# Batch Analysis (with example dataset)

### Nextclade df modification

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/data/Puja/All/Batch101/FASTQForAnalysis/SentinelHN/Batch101 - NextcladeSHN.csv')
df = df.rename(columns={'seqName':'sampleId'})
df = df[['sampleId', 'clade', 'lineage','Nextclade_pango', 'partiallyAliased', 'clade_nextstrain', 'clade_who', 'clade_display', 'qc.overallScore', 'totalSubstitutions', 'totalDeletions', 'aaSubstitutions', 'aaDeletions', 'aaInsertions']]
# df.columns
df

### Pangolin df modification

In [ ]:
df1 = pd.read_csv('/data/Puja/All/Batch100/SentinelHN/lineage_report.csv')
df1 = df1.rename(columns={'taxon':'sampleId'})
df1 = df1[['sampleId', 'lineage']]
df1

## NP merge

In [ ]:
df_NP = pd.merge(df, df1, on='sampleId', how='inner')
df_NP

### Metadata df modification

In [ ]:
df2 = pd.read_csv('/data/Puja/All/Batch101/FASTQForAnalysis/SentinelHN/Batch101 - Copy of Metadata_SHN.csv')
df2 = df2.rename(columns={'Sample ID given by the submitting laboratory':'sampleId'})
df2

### NPMeta merge

In [ ]:
df_NPmeta = pd.merge(df, df2, on='sampleId', how='inner')
df_NPmeta

### NPMeta df motdification

In [ ]:
df_NPmeta.dropna(subset=['lineage'], inplace=True)

In [ ]:
df_NPmeta['aaDeletions'] = df_NPmeta['aaDeletions'].apply(lambda x: '' if pd.isna(x) else x)
df_NPmeta['aaSubstitutions'] = df_NPmeta['aaSubstitutions'].apply(lambda x: '' if pd.isna(x) else x)
df_NPmeta['aaSubstitutions'] = df_NPmeta['aaSubstitutions'].str.split(',')
df_NPmeta['aaDeletions'] = df_NPmeta['aaDeletions'].str.split(',')

In [ ]:
df_NPmeta['aaSubDel'] = df_NPmeta['aaSubstitutions'] + df_NPmeta['aaDeletions']

In [ ]:
df_NPmeta = df_NPmeta.drop(columns=['aaSubstitutions', 'aaDeletions'])
df_NPmeta

# Mutational Analysis

## Identifying unique lineages and generating their mutation count matrices to study mutation frequencies

In [ ]:
df_NPmeta['lineage'].unique()

In [ ]:
df_D = df_NPmeta[df_NPmeta['lineage'] == 'JN.1.11.1']
df_D

In [ ]:
# Initialize an empty DataFrame to store the counts
count_df = pd.DataFrame()

# Iterate over rows in the original DataFrame
for index, row in df_D.iterrows():
    sample_id = row['sampleId']
    aa_sub_del_list = row['aaSubDel']

    # Create a dictionary to store counts for each value in aaSubDel
    counts = {}
    for value in aa_sub_del_list:
        counts[value] = counts.get(value, 0) + 1

    # Create a DataFrame for the current sampleId
    sample_df = pd.DataFrame(counts, index=[sample_id])

    # Concatenate the sample DataFrame with the overall counts DataFrame
    count_df = pd.concat([count_df, sample_df], sort=False)

# Fill NaN values with 0 and reset index
count_df = count_df.fillna(0).reset_index()

# Rename the 'index' column to 'sampleId'
count_df = count_df.rename(columns={'index': 'sampleId'})

# Display the final DataFrame
count_df

In [ ]:
count_df.to_csv('/data/Puja/All/Batch100/SentinelHN/KP2.csv')

# Weekly and Monthly Analyses (with example dataset)

In [ ]:
df = pd.read_csv('/data/Puja/All/Singapore_India Trend - Sheet4.csv')
df

In [ ]:
# Count the number of unique lineages per week
unique_lineages_per_week = df.groupby(["Month", "Week"])["Lineage"].nunique()

# Count the number of unique lineages per month
unique_lineages_per_month = df.groupby("Month")["Lineage"].nunique()

# Print the results
print("Unique lineages per week:")
print(unique_lineages_per_week)

print("\nUnique lineages per month:")
print(unique_lineages_per_month)

In [ ]:
lineage_counts_per_week = df.groupby(["Week", "Lineage"]).size().reset_index(name='Count')
lineage_counts_per_month = df.groupby(["Month", "Lineage"]).size().reset_index(name='Count')

In [ ]:
lineage_counts_per_week.to_csv('countperweek.csv')
lineage_counts_per_month.to_csv('countpermonth.csv')

In [ ]:
merged_df = pd.merge(lineage_counts_per_month, lineage_counts_per_week, on='Lineage', how='inner')
merged_df

# Fasta manipulations

## Extract fasta ids from a fasta file

In [ ]:
def extract_lines(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if line.startswith('>'):
                outfile.write(line)

# Example usage
input_file = '/data/Puja/All/HN_combinedfasta.txt'  # Replace with your input file path
output_file = '/data/Puja/All/HN_ids.txt'  # Replace with your desired output file path

extract_lines(input_file, output_file)

## Fasta extaction using ids and a combined fasta file

In [ ]:
# List of IDs to match
ids_to_match = [
    "VWA275", "VWA276", "VWA277", "VWA278", "VWA279", "VWA280", "VWA281",
    "VWA282", "VWA283", "VWA285", "VWA286", "VWA287", "VWA288", "VWA289"
]

# Input and output file paths
input_fasta_file = '/data/Puja/All/batch85.fasta'
output_fasta_file = '/data/Puja/All/Batch85.fasta'

# Function to extract matching sequences
def extract_matching_sequences(input_file, output_file, ids):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        write_sequence = False
        for line in infile:
            if line.startswith('>'):
                # Check if the header contains any of the IDs
                header = line.strip().split()[0][1:]  # Remove '>' and split to get the first part
                if header in ids:
                    write_sequence = True
                    outfile.write(line)
                else:
                    write_sequence = False
            elif write_sequence:
                outfile.write(line)

# Call the function to extract sequences
extract_matching_sequences(input_fasta_file, output_fasta_file, ids_to_match)

# Wastewater

## Freyja Demixed Lineage df analysis

In [ ]:
df = pd.read_csv('/data/Puja/All/WWPreviousBatches/Waste_Water_Seq_Raw_Data_previous_batches.xlsx - Sheet2.csv')
df

In [ ]:
#check type here
type(df['lineages'][0])

In [ ]:
df['lineages'] = df['lineages'].apply(eval)
df['abundances'] = df['abundances'].apply(eval)
df['summarized'] = df['summarized'].apply(eval)

In [ ]:
df['lineages'] = df['lineages'].apply(lambda x: eval(x) if type(x)!= float else '')
df['abundances'] = df['abundances'].apply(lambda x: eval(x) if type(x)!= float else '')

In [ ]:
df['lineages'] = df['lineages'].astype(str)
df['abundances'] = df['abundances'].astype(str)

In [ ]:
df['lineages'] = df['lineages'].str.split(',')
df['abundances'] = df['abundances'].str.split(',')

In [ ]:
df['lineage (abundances)'] = df.apply(lambda row: [f"{lineage} ({abundance})" for lineage, abundance in zip(row['lineages'], map(float, row['abundances']))], axis=1)

In [ ]:
df.to_csv('/data/Puja/All/WWPreviousBatches/modifiedLineages.csv')

# Infection tracking on incremental data

In [ ]:
# groupby lineage for unique mutations
df_uniqueMutations = df_Rajasthan.groupby('lineage').agg(lambda x: list(x))
df_uniqueMutations['aaSubDel'] = df_uniqueMutations['aaSubDel'].apply(lambda x: set([item for sublist in x for item in sublist]))
df_uniqueMutations

## Mutations from outbreak table and samples

In [ ]:
# Amino acid and sample mutation strings
amino_acid_str = """T9I A104V A63T Q19E G204R Q229K R203K S413R A211D A2710T G1307S K1973R L3027F N2526S P3395H T4175I V3593F P314L R1315C T223I S84L A570V D405N D614G E554K F486P G142D G339H I332V K356T K417N L216F N501Y N764K P1143L P621S Q498R S371F S373P S375F S477N T478K V213G D3H P13L S135R T3090I T3255I F19L D796Y del144/144 G446S N440K N460K N481K N679K N969K P681R Q183H R158G R408S V127F Y505H I1566V D61L E484K H655Y L452W L455S N450D Q954H S939F T376A V445H del212/212 N211I del3675/3677 T842I del31/33 R3821K T2163I S50L del483/483 V1056L del69/70 T30A R403K"""
sample_mutation_str = """S939F V1056L G204R T19I E2271G Q183H S375F S3675- R346T S135R F3677- Q19E I332V L452W T842I R403K K417N A2710T H245N L212I N28- P13L S33- D61L G339H S373P N679K D614G V3593F V213G R21T P10S F456L N460K A29- N211- P3395H P621S A63T P681R R32- S477N K1973R G446S L455S R158G V445H L216F E31- D3H F19L V127F N440K N969K T3255I Q954H S50L A211D F157S S413R A570V K356T G1307S N450D T9I Q229K I1566V H655Y N2526S D405N D796Y E27- L3674- R203K E554K R1315C G142D T2163I T376A A104V N764K S371F P1143L L3027F G3676- T223I P314L H69- V70- A264D T478K"""

# Convert strings to lists
amino_acid_list = amino_acid_str.split()
sample_mutation_list = sample_mutation_str.split()

# Find unique mutations
unique_mutations = list(set(amino_acid_list) ^ set(sample_mutation_list))

print("Unique Mutations:", len(unique_mutations))

In [ ]:
amino_acid_mutations = [
    "A104V", "A211D", "A264D", "A2710T", "A570V", "A63T", "D3H", "D405N", "D614G",
    "D61L", "D796Y", "del144/144", "del212/212", "del25/27", "del31/33", "del3675/3677",
    "del483/483", "del69/70", "E484K", "E554K", "F157S", "F19L", "F486P", "G1307S",
    "G142D", "G204R", "G339H", "G446S", "H245N", "H655Y", "I1566V", "I332V", "K1973R",
    "K356T", "K417N", "L216F", "L24S", "L3027F", "L452W", "L455S", "N211I", "N2526S",
    "N440K", "N450D", "N460K", "N481K", "N501Y", "N679K", "N764K", "N969K", "P1143L",
    "P13L", "P314L", "P3395H", "P621S", "P681R", "Q19E", "Q229K", "Q498R", "Q954H",
    "R1315C", "R158G", "R203K", "R21T", "R3821K", "R403K", "R408S", "S135R", "S371F",
    "S373P", "S375F", "S413R", "S477N", "S50L", "S84L", "S939F", "T19I", "T2163I",
    "T223I", "T3090I", "T30A", "T3255I", "T376A", "T4175I", "T478K", "T842I", "T9I",
    "V1056L", "V1104L", "V127F", "V213G", "V3593F", "V445H", "Y505H"
]
sample_mutations = [
    "A104V", "A211D", "A2710T", "A570V", "A63T", "D3H", "D405N", "D614G", "D796Y",
    "E554K", "F157S", "F19L", "F3677-", "G1307S", "G142D", "G334S", "G339H", "G3676-",
    "H245N", "H655Y", "I1566V", "I332V", "I71T", "K1973R", "K356T", "K417N", "L149P",
    "L212I", "L3027F", "N211-", "N2526S", "N679K", "N764K", "N969K", "P10S", "P13L",
    "P314L", "P621S", "P681R", "Q19E", "Q954H", "R1315C", "R158G", "R3821K", "R403K",
    "R408S", "S135R", "S166G", "S3675-", "S371F", "S373P", "S375F", "S413R", "S939F",
    "T2163I", "T223I", "T3032I", "T3090I", "T3255I", "T34P", "T376A", "T4175I", "T842I",
    "T899I", "T9I", "V1056L", "V1104L", "V127F", "V213G", "V3593F"
]
unique_mutations = list(set(amino_acid_mutations) ^ set(sample_mutations))

print("Unique Mutations:", unique_mutations)

## Mutational Matrix

In [ ]:
# Define the data for BA.2 and BA.2.86
JN1_11_1_data = "E:T9I	M:A104V	M:A63T	M:D3H	M:Q19E	M:T30A	N:del31/31	N:del32/32	N:del33/33	N:G204R	N:P13L	N:Q229K	N:R203K	N:S413R	ORF1a:A211D	ORF1a:A2710T	ORF1a:del3675/3675	ORF1a:del3676/3676	ORF1a:del3677/3677	ORF1a:G1307S	ORF1a:K1973R	ORF1a:L3027F	ORF1a:N2526S	ORF1a:P3395H	ORF1a:R3821K	ORF1a:S135R	ORF1a:T2283I	ORF1a:T3090I	ORF1a:T3255I	ORF1a:T4175I	ORF1a:T842I	ORF1a:V1056L	ORF1a:V3593F	ORF1b:I1566V	ORF1b:P314L	ORF1b:R1315C	ORF1b:T2163I	ORF3a:T223I	ORF6:D61L	ORF7b:F19L	ORF8:S84L	S:A264D	S:A570V	S:D405N	S:D614G	S:D796Y	S:del144/144	S:del212/212	S:del25/25	S:del26/26	S:del27/27	S:del483/483	S:del69/69	S:del70/70	S:E484K	S:E554K	S:F157S	S:F456L	S:F486P	S:G142D	S:G339H	S:G446S	S:H245N	S:H655Y	S:I332V	S:K356T	S:K417N	S:L216F	S:L24S	S:L452W	S:L455S	S:N211I	S:N440K	S:N450D	S:N460K	S:N481K	S:N501Y	S:N679K	S:N764K	S:N969K	S:P1143L	S:P621S	S:P681R	S:Q498R	S:Q954H	S:R158G	S:R21T	S:R403K	S:R408S	S:S371F	S:S373P	S:S375F	S:S477N	S:S50L	S:S939F	S:T19I	S:T376A	S:T478K	S:V1104L	S:V127F	S:V213G	S:V445H	S:Y505H"
KP4_data = "E:T9I M:A104V M:A63T M:D3H M:Q19E N:G204R N:P13L N:Q229K N:R203K N:S413R ORF1a:A211D ORF1a:A2710T ORF1a:G1307S ORF1a:K1973R ORF1a:L3027F ORF1a:N2526S ORF1a:P3395H ORF1a:R3821K ORF1a:S135R ORF1a:T2283I ORF1a:T3090I ORF1a:T3255I ORF1a:T4175I ORF1a:T842I ORF1a:V1056L ORF1a:V3593F ORF1b:I1566V ORF1b:P314L ORF1b:R1315C ORF1b:T2163I ORF3a:T223I ORF6:D61L ORF7b:F19L ORF8:S84L S:A264D S:A570V S:D405N S:D614G S:del212/212 S:E484K S:E554K S:F157S S:F456L S:F486P S:G142D S:G339H S:G446S S:H245N S:H655Y S:I332V S:K356T S:K417N S:L216F S:L452W S:L455S S:N211I S:N440K S:N450D S:N460K S:N501Y S:N679K S:N764K S:N969K S:P1143L S:P621S S:P681R S:Q498R S:Q954H S:R158G S:R408S S:S371F S:S373P S:S375F S:S477N S:S50L S:S939F S:T376A S:T478K S:V1104L S:V127F S:V213G S:V445H S:Y505H S:N481K"
# Convert the data strings into lists
JN1_11_1_list = JN1_11_1_data.split()
KP4_list = KP4_data.split()

# Find the extra mutations in BA.2.86 compared to BA.2
extra_mutations = [mutation for mutation in JN1_11_1_list if mutation not in KP4_list]

# Print the extra mutations
print(extra_mutations)

# GISAID metadata search (sample scrap search wrt labs)

In [ ]:
df = pd.read_csv('/data/Puja/All/metadata_2024-04-14_04-01.tsv', sep='\t')
df

In [ ]:
df1 = df[df['country'] == 'India']
df1.columns

In [ ]:
df3 = df1[df1['location'] == 'Kolkata']
df4 = df[df['originating_lab'].str.contains('Apollo Kolkata')]
df4

In [ ]:
df3['originating_lab'].unique()

In [ ]:
df2['originating_lab'].unique()

In [ ]:
labs = df1['originating_lab'].unique()